# Data loading

# Python packages




In [2]:
import math, datetime, time, random
import numpy as np
import pandas as pd
from datetime import timedelta
import re
import math, datetime, time, random
import matplotlib.pyplot as plt
from datetime import datetime, timedelta


# Door open - inbound




In [5]:
inbound = pd.read_csv('inbound_loads.csv')
outbound = pd.read_csv('outbound_laods.csv')
ft = pd.read_csv('featuretrain.csv')

### Uitleg

1. Get all the minutes for the interval in between start and finish


In [31]:
def interval_min(start_date, end_date):
    dates_range = []
    while start_date <= end_date:
        dates_range.append(start_date) # or you can append str(start_date) if want the dates to be as strings
        start_date += timedelta(minutes=1)
    return dates_range

In [32]:
door = outbound.filter(['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], axis=1)
door.head(5)

,load_start_datetime,load_finish_datetime,dock_door_number
0,2019-01-04 04:16:18,2019-01-04 04:19:31,62.0
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,2019-01-09 16:33:01,2019-01-09 18:40:24,37.0
4,2019-03-11 20:40:35,2019-03-11 21:22:00,37.0


2. Get all the minutes intervals and put them in a new dataframe

In [33]:
door['load_finish_datetime'] = pd.to_datetime(door['load_finish_datetime'])
door['load_start_datetime'] = pd.to_datetime(door['load_start_datetime'])

door['load_minutes'] = np.nan

for index, row in door.iterrows():
    door['load_minutes'].iloc[index] = interval_min(door['load_start_datetime'].iloc[index], door['load_finish_datetime'].iloc[index])

C:\Users\suzan\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [34]:
door = door.drop_duplicates(subset=['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], keep="first")

3. Make a flat list from the 'load_minutes' column 

In [35]:
def flatten(l):
    return [item for sublist in l for item in sublist]

list_min = door['load_minutes'].tolist()
list_min = flatten(list_min)
type(list_min[0])

pandas._libs.tslibs.timestamps.Timestamp

4. Create new dataframe with each time as an individual row (instead of list in each row). And change type from timeFrame to Datetime

In [36]:
minutes = pd.DataFrame(list_min)
minutes.columns =['datetime_local']
#minutes['datetime_local'] = pd.to_datetime(minutes['datetime_local'], unit='s')
#minutes['datetime_local'] = pd.Series(minutes.datetime_local.dt.to_pydatetime(), dtype='O')

5. Create a new dataframe with datetime and count

In [37]:
test = minutes.value_counts().to_frame('counts')
#test = minutes.value_counts().rename_axis('datetime_local').to_frame('counts')
test

,counts
datetime_local,
2020-02-08 07:03:34,4
2020-02-08 05:45:34,4
2020-02-08 06:57:34,4
2020-02-08 05:50:34,4
2020-02-08 05:25:34,4
...,...
2020-01-11 07:15:59,1
2020-01-11 07:16:30,1
2020-01-11 07:16:36,1


Bit sketchy but works.
- Reset the index level of dataframe 'test' and then turn Timeframe type into datetime
- Do the same with dataframe 'ft', so reset index level (is string), then turn type to datetime.


In [38]:
test = test.reset_index(level=[0])
#type(test.index)
#type(test['datetime_local'][0])

test['datetime_local'] = pd.to_datetime(test['datetime_local'], unit='s')
test['datetime_local'] = pd.Series(test.datetime_local.dt.to_pydatetime(), dtype='O')

In [42]:
#ft = ft.reset_index(level=[0])
type(ft['datetime_UTC'][0])
ft.head()

,datetime_local,level_0,index,year,month,weekday,day,hour,minute,datetime,Relative Humidity,Temperature,datetime_UTC
0,2018-12-31 18:00:00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:00:00-06:00,50.37,53.6,2019-01-01 00:00:00
1,2018-12-31 18:05:00,1,1,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:05:00-06:00,50.37,53.6,2019-01-01 00:05:00
2,2018-12-31 18:10:00,2,2,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:10:00-06:00,50.37,53.6,2019-01-01 00:10:00
3,2018-12-31 18:15:00,3,3,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:15:00-06:00,50.37,53.6,2019-01-01 00:15:00
4,2018-12-31 18:20:00,4,4,NaN,NaN,NaN,NaN,NaN,NaN,2018-12-31 18:20:00-06:00,50.37,53.6,2019-01-01 00:20:00


In [45]:
ft['datetime_local'] = pd.to_datetime(ft['datetime_local'])
ft['datetime_local'] = pd.Series(ft.datetime_local.dt.to_pydatetime(), dtype='O')
type(ft.datetime_local[0])

datetime.datetime

- Set datetime_local as index again. (Now the type of both is the same, which is necessary for merging correctly).

In [46]:
ft.set_index('datetime_local', inplace=True)
test.set_index('datetime_local', inplace=True)

- Now merge ft and test dataframe, on the content of the datetime_local of the ft dataframe. 


In [47]:
feat_min = ft.merge(test, left_on='datetime_local', right_index=True, how='left').reset_index()

In [48]:
feat_min["counts"] = feat_min["counts"].fillna(0)

#Drop certain columns if necessary
#feat_min = feat_min.drop(['level_0_x', 'index_x','level_0_y', 'index_y'], axis=1)
#feat_min.to_csv("feat_door_open")

In [50]:
feat_min.set_index('datetime_local', inplace=True)
feat_min = feat_min.drop(['level_0', 'index','year', 'month','weekday', 'day', 'hour', 'minute', 'datetime'], axis=1)
feat_min = feat_min.drop(['Temperature', 'Relative Humidity', 'datetime_UTC'], axis=1)
feat_min


,counts
datetime_local,
2018-12-31 18:00:00,0.0
2018-12-31 18:05:00,0.0
2018-12-31 18:10:00,0.0
2018-12-31 18:15:00,0.0
2018-12-31 18:20:00,0.0
...,...
2021-12-31 17:35:00,0.0
2021-12-31 17:40:00,0.0
2021-12-31 17:45:00,0.0


In [51]:
feat_min.iloc[400]

counts    0.0
Name: 2019-01-02 00:40:00, dtype: float64

In [52]:
feat_min.to_csv("feature_train_outbound_door_open")

# Outbound

In [14]:
door = outbound.filter(['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], axis=1)
door.head(5)

,load_start_datetime,load_finish_datetime,dock_door_number
0,2019-01-04 04:16:18,2019-01-04 04:19:31,62.0
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,2019-01-09 16:33:01,2019-01-09 18:40:24,37.0
4,2019-03-11 20:40:35,2019-03-11 21:22:00,37.0


In [15]:
door['load_finish_datetime'] = pd.to_datetime(door['load_finish_datetime'])
door['load_start_datetime'] = pd.to_datetime(door['load_start_datetime'])

door['load_minutes'] = np.nan

for index, row in door.iterrows():
    door['load_minutes'].iloc[index] = interval_min(door['load_start_datetime'].iloc[index], door['load_finish_datetime'].iloc[index])
    
door = door.drop_duplicates(subset=['load_start_datetime', 'load_finish_datetime', 'dock_door_number'], keep="first")

C:\Users\suzan\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
list_min = door['load_minutes'].tolist()
list_min = flatten(list_min)

minutes = pd.DataFrame(list_min)
minutes.columns =['datetime_local']

In [17]:
test = minutes.value_counts().to_frame('counts')


In [18]:
test

,counts
datetime_local,
2020-02-08 07:03:34,4
2020-02-08 05:45:34,4
2020-02-08 06:57:34,4
2020-02-08 05:50:34,4
2020-02-08 05:25:34,4
...,...
2020-01-11 07:15:59,1
2020-01-11 07:16:30,1
2020-01-11 07:16:36,1


In [19]:
test = test.reset_index(level=[0])
#type(test.index)
#type(test['datetime_local'][0])

test['datetime_local'] = pd.to_datetime(test['datetime_local'], unit='s')
test['datetime_local'] = pd.Series(test.datetime_local.dt.to_pydatetime(), dtype='O')

In [29]:
#ft = ft.reset_index(level=[0])
type(ft['datetime_UTC'][0])
#ft.head()
#ft['datetime_local'] = pd.to_datetime(ft['datetime_local'], unit='s')
#ft['datetime_local'] = pd.Series(ft.datetime_local.dt.to_pydatetime(), dtype='O')
type(ft.datetime_local[0])

AttributeError: 'DataFrame' object has no attribute 'datetime_local'

In [26]:
ft.set_index('datetime_local', inplace=True)
test.set_index('datetime_local', inplace=True)

In [27]:
feat_min = ft.merge(test, left_on='datetime_local', right_index=True, how='left').reset_index()

ValueError: You are trying to merge on object and datetime64[ns] columns. If you wish to proceed you should use pd.concat

In [ ]:
feat_min["counts"] = feat_min["counts"].fillna(0)

#Drop certain columns if necessary
#feat_min = feat_min.drop(['level_0_x', 'index_x','level_0_y', 'index_y'], axis=1)
feat_min


In [ ]:
#feat_min.to_csv("feat_door_open_outbound")